In [ ]:
#already installed both libraries
import pandas as pd
import numpy as np

In [ ]:
df=pd.read_csv("/content/labeled_compounds.tsv" , sep='\t')

In [ ]:
df.head()

,Molecule ChEMBL ID,Smiles,EC50,Tanimoto_with_Rosi,Tanimoto_with_Pio,Average_Similarity,Label
0,CHEMBL121,CN(CCOc1ccc(CC2SC(=O)NC2=O)cc1)c1ccccn1,300.0,1.0000,0.5246,0.7623,1
1,CHEMBL121,CN(CCOc1ccc(CC2SC(=O)NC2=O)cc1)c1ccccn1,320.0,1.0000,0.5246,0.7623,1
2,CHEMBL595,CCc1ccc(CCOc2ccc(CC3SC(=O)NC3=O)cc2)nc1,240.0,0.5246,1.0000,0.7623,1
3,CHEMBL121,CN(CCOc1ccc(CC2SC(=O)NC2=O)cc1)c1ccccn1,100.0,1.0000,0.5246,0.7623,1
4,CHEMBL121,CN(CCOc1ccc(CC2SC(=O)NC2=O)cc1)c1ccccn1,657.0,1.0000,0.5246,0.7623,1


In [ ]:
# taking id, smiles,label column for cleaning

# Columns want to keep
columns_to_keep = ['Molecule ChEMBL ID', 'Smiles', 'Label']

# Select these columns from the dataframe
cleaned_df = df[columns_to_keep]

# Save to TSV file
cleaned_df.to_csv('cleaned_data.tsv', sep='\t', index=False)

print("Cleaned data saved to 'cleaned_data.tsv'")


Cleaned data saved to 'cleaned_data.tsv'


In [ ]:
df =pd.read_csv("/content/cleaned_data.tsv" , sep='\t')
df.head()

,Molecule ChEMBL ID,Smiles,Label
0,CHEMBL121,CN(CCOc1ccc(CC2SC(=O)NC2=O)cc1)c1ccccn1,1
1,CHEMBL121,CN(CCOc1ccc(CC2SC(=O)NC2=O)cc1)c1ccccn1,1
2,CHEMBL595,CCc1ccc(CCOc2ccc(CC3SC(=O)NC3=O)cc2)nc1,1
3,CHEMBL121,CN(CCOc1ccc(CC2SC(=O)NC2=O)cc1)c1ccccn1,1
4,CHEMBL121,CN(CCOc1ccc(CC2SC(=O)NC2=O)cc1)c1ccccn1,1


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5305 entries, 0 to 5304
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Molecule ChEMBL ID  5305 non-null   object
 1   Smiles              5305 non-null   object
 2   Label               5305 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 124.5+ KB


In [ ]:
label_counts = df['Label'].value_counts()
print(label_counts)

Label
0    5081
1     224
Name: count, dtype: int64


In [ ]:
# To see the duplicate smiles
duplicate_smiles = df[df.duplicated(subset=['Smiles'], keep=False)]
duplicate_counts = duplicate_smiles['Smiles'].value_counts()
print("Duplicate Smiles entries and their counts:")
print(duplicate_counts)


Duplicate Smiles entries and their counts:
Smiles
CN(CCOc1ccc(CC2SC(=O)NC2=O)cc1)c1ccccn1                                     156
CCc1ccc(CCOc2ccc(CC3SC(=O)NC3=O)cc2)nc1                                      43
Cc1cc(SCc2sc(-c3ccc(C(F)(F)F)cc3)nc2C)ccc1OCC(=O)O                           21
Cc1c(C)c2c(c(C)c1O)CCC(C)(COc1ccc(CC3SC(=O)NC3=O)cc1)O2                      20
Cc1cccc(Nc2cc(Cl)nc(SCC(=O)O)n2)c1C                                          20
                                                                           ... 
CN(CCOc1ccc(CC(Nc2ccccc2/C(=N/N)c2ccccc2)C(=O)O)cc1)c1ccccn1                  2
Cc1oc(-c2ccccc2)nc1CCOc1ccc(CC(Nc2ccccc2C(=O)c2cccc(CN(C)C)c2)C(=O)O)cc1      2
CCC(Cc1ccc(OC)c(CNC(=O)c2ccc(C(F)(F)F)cc2F)c1)C(=O)O                          2
CN(CCOc1ccc(CC(Nc2cscc2C(=O)c2ccccc2)C(=O)O)cc1)c1nc2ccccc2o1                 2
O=S(=O)(CS(=O)(=O)Oc1cc(Cl)cc(Cl)c1)Oc1cc(Cl)cc(Cl)c1                         2
Name: count, Length: 430, dtype: int64


In [ ]:
#To see the duplicate chembl id

chembl_column = 'Molecule ChEMBL ID'

# Count how many times each ChEMBL ID appears
chembl_counts = df[chembl_column].value_counts().reset_index()
chembl_counts.columns = [chembl_column, 'Repeat_Count']

# Filter only duplicates (those appearing more than once)
duplicate_chembl = chembl_counts[chembl_counts['Repeat_Count'] > 1]


# Print summary
print(duplicate_chembl)
print(f"Number of unique ChEMBL IDs that are duplicated: {duplicate_chembl.shape[0]}")

    Molecule ChEMBL ID  Repeat_Count
0            CHEMBL121           156
1            CHEMBL595            43
2          CHEMBL38943            21
3         CHEMBL295416            20
4            CHEMBL408            20
..                 ...           ...
425       CHEMBL412371             2
426       CHEMBL363805             2
427        CHEMBL80791             2
428      CHEMBL1801740             2
429      CHEMBL4470753             2

[430 rows x 2 columns]
Number of unique ChEMBL IDs that are duplicated: 430


In [ ]:
# checking , together the duplicates are same
import pandas as pd

# Define columns
chembl_col = 'Molecule ChEMBL ID'
smiles_col = 'Smiles'

# Count how many times each (ChEMBL_ID, Smiles) pair appears
pair_counts = df.groupby([chembl_col, smiles_col]).size().reset_index(name='Repetition')

# Filter only the pairs that appear more than once
duplicate_pairs = pair_counts[pair_counts['Repetition'] > 1]

# Save to a TSV file
duplicate_pairs.to_csv('repeated_chemblid_smile.tsv', sep='\t', index=False)

# Print summary
print(duplicate_pairs)
print(f"Number of duplicate (Molecule ChEMBL ID, SMILES) pairs: {duplicate_pairs.shape[0]}")


     Molecule ChEMBL ID                                             Smiles  \
3            CHEMBL1017  CCCc1nc2c(C)cc(-c3nc4ccccc4n3C)cc2n1Cc1ccc(-c2...   
5          CHEMBL104246  Cc1oc(-c2ccccc2)nc1CCOc1ccc(C[C@@H](C(=O)O)N2C...   
8          CHEMBL104850  Cc1oc(-c2ccccc2)nc1CCOc1ccc(C[C@@H](C(=O)O)n2c...   
10         CHEMBL106666  COC(=O)c1ccccc1N[C@@H](Cc1ccc(OCCc2nc(-c3ccccc...   
11         CHEMBL107367  Cc1oc(-c2ccccc2)nc1CCOc1ccc(C[C@H](Nc2ccccc2C(...   
...                 ...                                                ...   
4318        CHEMBL87307  CCO[C@@H](Cc1ccc(OCC=C(c2ccc(Br)cc2)c2ccc(Br)c...   
4319        CHEMBL87380  CCO[C@@H](Cc1ccc(OCC=C(c2ccc(C)cc2)c2ccc(C)cc2...   
4322        CHEMBL89152  CCO[C@@H](Cc1ccc(OC/C=C(\c2ccccc2)c2ccc(Br)cc2...   
4326        CHEMBL94306  CCCc1cc(Oc2ccccc2)ccc1OCCCOc1cccc(C2SC(=O)NC2=...   
4335          CHEMBL981          CC(C)(Oc1ccc(C(=O)c2ccc(Cl)cc2)cc1)C(=O)O   

      Repetition  
3             13  
5              2  
8     

Cleaning the duplicates

In [ ]:
# drop the duplicates - chembl id and smile

# Load your dataset
df = pd.read_csv('cleaned_data.tsv', sep='\t')

# Drop duplicate rows where both ChEMBL_ID and Smiles are same
df_cleaned = df.drop_duplicates(subset=['Molecule ChEMBL ID', 'Smiles'], keep='first')

# Save cleaned file
df_cleaned.to_csv('cleaned_data.tsv', sep='\t', index=False)

print(f"✅ Cleaned file saved as 'cleaned_chembl_smiles.tsv'")
print(f"Before cleaning: {len(df)} rows")
print(f"After cleaning:  {len(df_cleaned)} rows")
print(f"Duplicates removed: {len(df) - len(df_cleaned)}")


✅ Cleaned file saved as 'cleaned_chembl_smiles.tsv'
Before cleaning: 5305 rows
After cleaning:  4336 rows
Duplicates removed: 969


In [ ]:
df=pd.read_csv('/content/cleaned_data.tsv' , sep='\t')
df.head()

,Molecule ChEMBL ID,Smiles,Label
0,CHEMBL121,CN(CCOc1ccc(CC2SC(=O)NC2=O)cc1)c1ccccn1,1
1,CHEMBL595,CCc1ccc(CCOc2ccc(CC3SC(=O)NC3=O)cc2)nc1,1
2,CHEMBL59132,Cc1oc(-c2ccccc2)nc1CCOc1ccc(CC2SC(=O)NC2=O)cc1,1
3,CHEMBL4127972,O=C1NC(=O)C(Cc2ccc(OCCOC(c3ccccc3)c3ccccc3)cc2)S1,1
4,CHEMBL4293295,CCc1ccc([C@H](O)COc2ccc(CC3SC(=O)NC3=O)cc2)nc1,1


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4336 entries, 0 to 4335
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Molecule ChEMBL ID  4336 non-null   object
 1   Smiles              4336 non-null   object
 2   Label               4336 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 101.8+ KB


In [ ]:
df = pd.read_csv('/content/cleaned_data.tsv' , sep='\t')
label_counts = df['Label'].value_counts()
print(label_counts)

Label
0    4314
1      22
Name: count, dtype: int64


Checking any duplicates smiles are present after cleaning

In [ ]:

df = pd.read_csv('/content/cleaned_data.tsv' , sep='\t')
# Check for duplicates
duplicate_mask = df['Smiles'].duplicated(keep=False)

# How many duplicated rows
num_duplicates = duplicate_mask.sum()

# How many unique SMILES are repeated
num_unique_duplicates = df['Smiles'].value_counts()[df['Smiles'].value_counts() > 1].count()

print(f"Total duplicated rows: {num_duplicates}")
print(f"Number of unique SMILES repeated: {num_unique_duplicates}")


Total duplicated rows: 0
Number of unique SMILES repeated: 0


Checking all the smiles are valid, by using rdkit

In [ ]:

! pip install rdkit
from rdkit import Chem
df = pd.read_csv('/content/cleaned_data.tsv' , sep='\t')
# Function to check validity
def is_valid_smiles(smi):
    return Chem.MolFromSmiles(smi) is not None

# Apply and get counts
valid_counts = df['Smiles'].apply(is_valid_smiles).value_counts()

print("SMILES validity summary:")
print(valid_counts)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.1/36.1 MB 35.5 MB/s eta 0:00:00
✅ SMILES validity summary:
Smiles
True    4336
Name: count, dtype: int64
